In [1]:
%pylab inline
from enum import Enum

Populating the interactive namespace from numpy and matplotlib


# Algebra Solver

## Problem statement

Solve `1/2*10*(5+2)/3`.

We should follow PEMDAS:

1. Parenthesis
2. Exponents
3. Multiplication/Division
4. Addition/Subtraction

## Parsing (Tokenizing)

### Token class

This will hold the type of token and the value of the token. There are different types of tokens:

**Types:** `symbol`, `function`, `term`, `constant`

In [172]:
class Token():
    def __init__(self, type, value):
        self.type = type
        self.value = value
    
    def __repr__(self):
        return "['%s': '%s']" % (self.type, self.value)

### Tokenizer class

This is engine for the parser, all the character/digits will be converted into a list of tokens which can be used in lexical analysis later on.

**Whitespace characters:** `<space>`, `\n`, `\r`, `\t\`.

**Supported symbols:** `+`, `-`, `*`, `/`, `=`, `^`, `(`, `)`, `,`.

**Supported functions:** `sin`, `asin`, `cos`, `acos`, `tan`, `atan`, `log`, `ln`.

In [171]:
import string

class Tokenizer():
    whitespace = [' ', '\r', '\n', '\t']
    symbols = {'+':'add', '-':'subtract', '*':'multiply', '/':'divide', '=':'equal',
               '^':'power', '(':'left-parenthesis', ')':'right-parenthesis', ',':'comma'}
    functions = ['sin', 'asin', 'cos', 'acos', 'tan', 'atan', 'log', 'ln']
    
    def __init__(self, expression):
        self.expression = expression
        self.index = 0
        self.tokens = []
        self.tokenize()
        
    def next(self, skip = 1):
        self.index += skip
        
    def end(self):
        if self.index >= len(self.expression): return True
        return False
    
    def nextCharacter(self):
        return self.expression[self.index:self.index+1]
    
    def eatWhitespace(self):
        while self.nextCharacter() in self.whitespace:
            self.next()
            
    def eatDigit(self):
        digit = ''
        while self.nextCharacter().isdigit():
            digit += self.nextCharacter()
            self.next()
        return digit
    
    def eatWord(self):
        word = ''
        while self.nextCharacter() in string.ascii_lowercase + string.ascii_uppercase:
            word += self.nextCharacter()
            self.next()
        return word
            
    def tokenize(self):
        while(not self.end()):
            self.eatWhitespace()
            if self.nextCharacter() in self.symbols:
                token = Token('symbol', self.symbols[self.nextCharacter()])
                self.tokens.append(token)
                self.next()
            elif self.nextCharacter() in string.ascii_lowercase + string.ascii_uppercase:
                word = self.eatWord()
                if word in self.functions:
                    token = Token('function', word)
                elif len(word) == 1:
                    token = Token('term', word)
                else:
                    raise ValueError('Invalid character at index %i near %s'  % (self.index, self.expression[self.index-5:self.index+5]))
                self.tokens.append(token)
            elif self.nextCharacter().isdigit():
                digit = self.eatDigit()
                token = Token('constant', digit)
                self.tokens.append(token)

We can use the tokenizer with any mathematical expression, like:

In [176]:
tokenizer = Tokenizer('(2g + 36x * ln(2) * acos(6)A * 2) / 2 tan(5) = log(23,5) + 3')

This is example of the output of the tokenizer. The result is a list of `Token` objects.

In [177]:
tokenizer.tokens

[['symbol': 'left-parenthesis'],
 ['constant': '2'],
 ['term': 'g'],
 ['symbol': 'add'],
 ['constant': '36'],
 ['term': 'x'],
 ['symbol': 'multiply'],
 ['function': 'ln'],
 ['symbol': 'left-parenthesis'],
 ['constant': '2'],
 ['symbol': 'right-parenthesis'],
 ['symbol': 'multiply'],
 ['function': 'acos'],
 ['symbol': 'left-parenthesis'],
 ['constant': '6'],
 ['symbol': 'right-parenthesis'],
 ['term': 'A'],
 ['symbol': 'multiply'],
 ['constant': '2'],
 ['symbol': 'right-parenthesis'],
 ['symbol': 'divide'],
 ['constant': '2'],
 ['function': 'tan'],
 ['symbol': 'left-parenthesis'],
 ['constant': '5'],
 ['symbol': 'right-parenthesis'],
 ['symbol': 'equal'],
 ['function': 'log'],
 ['symbol': 'left-parenthesis'],
 ['constant': '23'],
 ['symbol': 'comma'],
 ['constant': '5'],
 ['symbol': 'right-parenthesis'],
 ['symbol': 'add'],
 ['constant': '3']]